In [ ]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/whoswastika/datascienceproject.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="whoswastika"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6b150e5f4317b1a63054fb99a38c885c512a6356"

In [2]:
import os
%pwd

'c:\\Users\\Admin\\OneDrive\\Desktop\\MLOPS\\datascienceproject\\research'

In [3]:
os.chdir("../")
%pwd

'c:\\Users\\Admin\\OneDrive\\Desktop\\MLOPS\\datascienceproject'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path : Path
    all_params : Path
    metric_file_name: Path
    target_column : str
    mlflow_uri : str
   

In [5]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories,save_json

In [6]:
class ConfigurationManager:
    def __init__ (
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config= read_yaml(config_filepath)
        self.params= read_yaml(params_filepath)
        self.schema= read_yaml(schema_filepath)
    
    
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config=self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config= ModelEvaluationConfig(
            root_dir= Path(config.root_dir),
            test_data_path= Path(config.test_data_path),
            model_path = Path(config.model_path),
            all_params = params,
            metric_file_name= Path(config.metric_file_name),
            target_column = schema.name,
            mlflow_uri = os.getenv("MLFLOW_TRACKING_URI")
        )
        return model_evaluation_config
        
        # model_evaluation_config= ModelEvaluationConfig(
        #     root_dir= config.root_dir,
        #     test_data_path= config.test_data_path,
        #     model_path = config.model_path,
        #     all_params = params,
        #     metric_file_name= config.metric_file_name,
        #     target_column = schema.name,
        #     mlflow_uri ="https://dagshub.com/whoswastika/datascienceproject.mlflow"
        # )
         
        
        
        
        
        
        

In [7]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [8]:
class ModelEvaluation:
    def __init__(self,config : ModelEvaluationConfig):
        self.config = config
        
    def eval_metrics(Self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        test_x = test_data.drop(self.config.target_column, axis=1)
        test_y = test_data[self.config.target_column]
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)
            
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            scores = {"rsme": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)
            mlflow.log_params(self.config.all_params)
            
            # print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (self.config.all_params.alpha, self.config.all_params.l1_ratio))
            # print("  RMSE: %s" % rmse)
            # print("  MAE: %s" % mae)
            # print("  R2: %s" % r2)
            
            # mlflow.log_param("alpha", self.config.all_params.alpha)
            # mlflow.log_param("l1_ratio", self.config.all_params.l1_ratio)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [9]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
    
except Exception as e:
    raise e

[2025-09-07 02:20:17,272: common: yaml file:config\config.yaml loaded successfully]
[2025-09-07 02:20:17,307: common: yaml file:params.yaml loaded successfully]
[2025-09-07 02:20:17,328: common: yaml file:schema.yaml loaded successfully]
[2025-09-07 02:20:17,336: common: created directory at: artifacts_root]
[2025-09-07 02:20:17,340: common: created directory at: artifacts/model_evaluation]
[2025-09-07 02:20:18,259: common: json file saved at:artifacts\model_evaluation\metrics.json]


2025/09/07 02:20:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/09/07 02:20:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
